In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [ ]:
import seaborn as sns
import chaospy as cp
import monte_carlo as mc
import pandas as pd

In [ ]:
from vewk3_models import *
#from scipy_vewk3 import *

In [ ]:
# Taken from Segers et al Hypertension 2000
closed_loop_base_pars = {'C_ao': 1.13, 
             'E_max': 1.5, 
             'E_min': 0.03,
             'R_mv': 0.006,
             'R_sys': 1.11,
             'T': 0.85,
             'Z_ao': 0.033,
             't_peak': 0.3,
             'C_sv': 11,
             'V_tot': 300
            }


ve_closed = VaryingElastance()
ve_closed.set_pars(**closed_loop_base_pars)
var_dict, t_eval, scipy_sol = solve_to_steady_state(ve_closed, n_cycles=15)
ret_dict = calc_summary(var_dict)
print(ret_dict)
plot_pressures(var_dict)

# Sample to compare with Stergiopolus 1999 Open Loop

## One factor at a time analysis

In [ ]:
def ofat(ve_model, params, dev, sigma_x):
    ve_model.set_pars(**params)
    var_dict, t_eval, scipy_sol = solve_to_steady_state(ve_model)
    ret_dict = calc_summary(var_dict)
    df0 = pd.DataFrame(ret_dict, index=[0], )
    input_names = sorted(params.keys())
    data_array = []
    dydx = pd.DataFrame()
    pct_sensitivities = pd.DataFrame()
    sigma_normalized_sens = pd.DataFrame()
    total_variance = 0*df0
    for par in input_names:
        pars = dict(params)
        pars[par] = (1+dev)*pars[par]
        ve_model.set_pars(**pars)
        var_dict, t_eval, scipy_sol = solve_to_steady_state(ve_model)
        ret_dict = calc_summary(var_dict)
        #dydx.loc[par][df0.columns] = (pd.DataFrame(ret_dict, index=[par,]) - df0.loc[0]) #/(dev*params[par])
        dydx = dydx.append( (pd.DataFrame(ret_dict, index=[par,]) - df0.loc[0])/(dev*params[par]))
        
        partial_variance = (dydx.loc[par]*sigma_x.loc[0,par])**2
        total_variance = total_variance + partial_variance
        partial_variance.name = par
        sigma_normalized_sens =  sigma_normalized_sens.append(partial_variance)
        
        pct_sens  = 100*dydx.loc[par]*(params[par]/df0.loc[0])
        pct_sens.name = par
        pct_sensitivities = pct_sensitivities.append(pct_sens)
        data_array.append(ret_dict)
    df = pd.DataFrame(data_array, index=input_names)
    sigma_normalized_sens = sigma_normalized_sens/total_variance.loc[0]
    return dydx, pct_sensitivities, sigma_normalized_sens

In [ ]:
closed_loop_sa_pars = {'C_ao': 1.13, 
             'E_max': 1.5, 
             'E_min': 0.03,
             'C_sv': 10,
             'R_sys': 1.11,
             'T': 0.85,
             'Z_ao': 0.033,
             'V_tot':300
            }

dev = 0.1
ve_closed = VaryingElastance()
ve_closed.set_pars(**closed_loop_base_pars)
finite_difference_step = np.sqrt(1e-7) #1e-3 # sqrt(accuracy of ode)
sigma_x = np.sqrt(pd.DataFrame(closed_loop_sa_pars, index=[0,])**2 * dev / 6)
res = ofat(ve_closed, closed_loop_sa_pars, finite_difference_step, sigma_x)
closed_loop_ofat_results = dict(dydx=res[0], pct_sens=res[1], sigma_sens=res[2])

## Global 10% uncertainty analysis

### Closed loop

In [ ]:
closed_loop_sa_pars = {'C_ao': 1.13, 
             'E_max': 1.5, 
             'E_min': 0.03,
             'C_sv': 10,
             'R_sys': 1.11,
             'T': 0.85,
             'Z_ao': 0.033,
             #'V_tot':300,
             't_peak':0.3,
             'R_mv':0.006
            }

dev = 0.1
input_names = sorted(closed_loop_sa_pars.keys())

dists = []
for par in input_names:
    par_nominal = closed_loop_sa_pars[par]
    par_range = np.array([1-dev, 1+dev])*par_nominal
    dists.append(cp.Uniform(*par_range))

joint_dist = cp.J(*dists)

In [ ]:
def vectorized_rapper_for_uqsa(samples):
    results = []
    for sample in samples:
        sample_pars = dict(closed_loop_base_pars)
        for idx, par in enumerate(input_names):
            sample_pars[par] = sample[idx]
        ve_closed = VaryingElastance()
        ve_closed.set_pars(**sample_pars)
        var_dict, t_eval, scipy_sol = solve_to_steady_state(ve_closed)
        #results.append(var_dict)
        ret_dict = calc_summary(var_dict)
        results.append(ret_dict)
    return results

In [ ]:
Ns=5000
A, B, C = mc.generate_sample_matrices(Ns, joint_dist)
dataA, dataB, dataC = mc.evaluate_samples(vectorized_rapper_for_uqsa, A, B, eval_mode="parallel")

In [ ]:
dfA = pd.DataFrame(dataA)
dfB = pd.DataFrame(dataB)
#dfsC = [pd.DataFrame(data) for data in dataC] # doesn't work
dfsC = [pd.DataFrame(list(data)) for data in dataC]

In [ ]:
dfA["PP"] = dfA["P_sys"] - dfA["P_dia"]
dfB["PP"] = dfB["P_sys"] - dfB["P_dia"]
#del dfA["stroke_work_int"]
#del dfB["stroke_work_int"]
for df in dfsC:
    df["PP"] = df["P_sys"] - df["P_dia"]
    #del df["stroke_work_int"]

In [ ]:
y_a = dfA.values
y_b = dfB.values
y_c = np.array([np.stack(df.values) for df in dfsC])
s_i, s_t_i = mc.calculate_sensitivity_indices(y_a, y_b, y_c)

df_Sm = pd.DataFrame(index=input_names, columns=dfA.columns, data=s_i)
df_St = pd.DataFrame(index=input_names, columns=dfA.columns, data=s_t_i)


In [ ]:
for ns in [2500, 3500, 4500]:
#for ns in [500, 750, 900, 950]:
    s_i_check, s_t_i_check = mc.calculate_sensitivity_indices(y_a[0:ns], y_b[0:ns], y_c[:,0:ns])
    df_Sm_err = pd.DataFrame(index=input_names, columns=dfA.columns, data=s_i - s_i_check)
    df_St_err = pd.DataFrame(index=input_names, columns=dfA.columns, data=s_i - s_t_i_check)
    print("Samples {} max global error: {}, {}".format(ns,
                                                df_Sm_err.abs().max().max(),
                                                df_St_err.abs().max().max()))
    print("Max error by variable")
    print("max S_m error", df_Sm_err.abs().max())
    print("max S_t error", df_St_err.abs().max())

In [ ]:
## dy/dx *(x/y) * 100 -> percent change in y per 100 percent change in x
df_X_pct_change = pd.DataFrame((cp.Std(joint_dist)/cp.E(joint_dist)), index=input_names)
display(100 * np.sqrt(np.abs(df_Sm)).div(df_X_pct_change[0], axis=0) * (dfA.std()/dfA.mean()))
# percent variation in Y  due to  100% variation in X 
display(closed_loop_ofat_results["pct_sens"])


In [ ]:
display(df_Sm)
display(closed_loop_ofat_results["sigma_sens"])
display(np.max(df_Sm - closed_loop_ofat_results["sigma_sens"]))

In [ ]:
closed_loop_gsa =  dict(input_names=input_names, df_Sm=df_Sm, df_St=df_St, dfA=dfA)
%store closed_loop_gsa

In [ ]:
df_Sm = pd.DataFrame(index=input_names, columns=dfA.columns, data=s_i)
colors = ["#feedde", 
          "#fdd0a2",
"#fdae6b",
"#fd8d3c",
"#f16913",
"#d94801",
"#8c2d04"]

def highlight_funct(x):
    if x > 0.5:
        ret_val = 'background-color: %s' % colors[0]
    elif x>0.4:
        ret_val = 'background-color : %s' % colors[1]
    elif x>0.3:
        ret_val = 'background-color : %s' % colors[2]
    elif x>0.2:
        ret_val = 'background-color : %s' % colors[3]
    elif x>0.1:
        ret_val = 'background-color : %s' % colors[4]
    elif x>0.05:
        ret_val = 'background-color : %s' % colors[5]
    else:    
        ret_val =  'background-color: %s' % colors[6]
    return ret_val
        
    
df_Sm.style.applymap(highlight_funct)

In [ ]:
df_St = pd.DataFrame(index=input_names, columns=dfA.columns, data=s_t_i)
df_St.style.applymap(highlight_funct)

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(9,9))
sns.heatmap(df_Sm, ax=ax[0])# center=0)
sns.heatmap(df_St, ax=ax[1])# center=0)

In [ ]:
if Ns == 1000:
    var = "P_ao"
    y_a = np.stack(dfA[var].values)
    y_b = np.stack(dfB[var].values)
    y_c = np.array([np.stack(df[var].values) for df in dfsC])
    s_i, s_t_i = mc.calculate_sensitivity_indices(y_a, y_b, y_c)
    #print(s_i)
    plt.figure()
    for idx, s_t in enumerate(s_t_i):
        h = plt.plot(s_t, label=input_names[idx])
        plt.plot(s_i[idx], '--', color=h[0].get_color())
    plt.legend(loc="upper left", bbox_to_anchor=(1,1))
    plt.tight_layout()

In [ ]:
if Ns == 500:
    var = "P_ao"
    # y_a = np.stack(dfA[var].values).mean(axis=1)
    # y_b = np.stack(dfB[var].values).mean(axis=1)
    # y_c = np.array([np.stack(df[var].values) for df in dfsC]).mean(axis=1)
    y_a = np.stack(dfA[var].values)
    y_b = np.stack(dfB[var].values)
    y_c = np.array([np.stack(df[var].values) for df in dfsC])
    s_i, s_t_i = mc.calculate_sensitivity_indices(y_a, y_b, y_c)
    #print(s_i)
    plt.figure()
    for idx, s_t in enumerate(s_t_i):
        h = plt.plot(s_t, label=input_names[idx])
        plt.plot(s_i[idx], '--', color=h[0].get_color())
    plt.legend(loc="upper left", bbox_to_anchor=(1,1))
    plt.tight_layout()

In [ ]:
var = "Q_lvao"
y_a = np.stack(dfA[var].values).mean(axis=1)
y_b = np.stack(dfB[var].values).mean(axis=1)
y_c = np.array([np.stack(df[var].values) for df in dfsC]).mean(axis=2)
s_i, s_t_i = mc.calculate_sensitivity_indices(y_a, y_b, y_c)
print(input_names)
print(s_i)
print(s_t_i)
plt.figure()
plt.plot(s_t_i)
plt.gca().set(xticks=np.arange(len(input_names)))
plt.gca().set(xticklabels=input_names)

In [ ]:
var = "P_ao"
y_a = np.stack(dfA[var].values).max(axis=1)
y_b = np.stack(dfB[var].values).max(axis=1)
y_c = np.array([np.stack(df[var].values) for df in dfsC]).max(axis=2)
s_i, s_t_i = mc.calculate_sensitivity_indices(y_a, y_b, y_c)
print(input_names)
print(s_i)
print(s_t_i)
plt.figure()
plt.plot(s_t_i)
plt.gca().set(xticks=np.arange(len(input_names)))
plt.gca().set(xticklabels=input_names)

In [ ]:
var = "P_ao"
y_a = np.stack(dfA[var].values).min(axis=1)
y_b = np.stack(dfB[var].values).min(axis=1)
y_c = np.array([np.stack(df[var].values) for df in dfsC]).min(axis=2)
s_i, s_t_i = mc.calculate_sensitivity_indices(y_a, y_b, y_c)
print(input_names)
print(s_i)
print(s_t_i)
plt.figure()
plt.plot(s_t_i)
plt.gca().set(xticks=np.arange(len(input_names)))
plt.gca().set(xticklabels=input_names)